## Word2Vec

In [35]:
#!pip install gensim

In [36]:
from gensim.models.word2vec import Word2Vec
import pandas as pd
import gensim
from konlpy.tag import Okt

In [37]:
#  Nsmc 데이터 로드
train_df = pd.read_csv('./data/ratings_train.csv')
train_df.dropna(inplace=True)
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


기본 전처리 과정

1. 한글만 남김 (특수 기호 제거)
2. 명사만 추출

In [38]:
# 정규식 이용하여 한글만 추출 : "[^ㄱ-ㅎㅏ-ㅣ가-힣]"

train_df['document'] = train_df['document'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]+', " ")
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [39]:
# 불용어 정의
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '을', '를', '는', '으로']

In [40]:
# 형태소 분석 및 불용어 제거하여 데이터 전처리하기
# 1. 문서 하나씩 읽어오기 (50개만)
# 2. 문서에 okt 형태소 분석 적용 (morphs함수, stem=True 변경)
# 3. 불용어 제거
# 4. tokenized_words 리스트(문서 별 형태소 추출): [[words, words, words], [words, words, words]]
# 5. sentence_nouns 리스트(문서 별 명사 리스트): [[noun, noun, noun, noun], [noun, noun, noun, noun]]
okt = Okt()
tokenized_words, sentence_nouns = [] , []


for sentence in train_df['document'][:50]:
    # 형태소 분석 결과
    tokenized_words = okt.morphs(sentence, stem=True) # stem= True -> 어간추출
    # 불용어 제거
    stopwords_removed_sentence = [word for word in tokenized_words if not word in stopwords]
    tokenized_words.append(stopwords_removed_sentence)

    # 명사 추출 결과
    nouns = okt.nouns(sentence)
    sentence_nouns.append(nouns)
# 딥러닝 max length : 문서의 최대길이로 맞춰서 픽스하는데
# 이 길이보다 작으면 패딩(0)으로 채움

In [41]:
# import pandas as pd
# from konlpy.tag import Okt
# import re

# # Nsmc 데이터 로드
# train_df = pd.read_csv('./data/ratings_train.csv')
# train_df.dropna(inplace=True)

# # 정규식을 이용하여 한글만 추출
# def extract_korean(text):
#     korean_pattern = re.compile('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]+')
#     return korean_pattern.sub("", text)

# # 데이터 샘플링 및 한글 추출
# sample_data = train_df['document'][:50].astype(str)
# korean_sample_data = sample_data.apply(extract_korean)

# # 불용어 정의
# stopwords = ['의', '가', '이', '은', '들', '는', '좀', '을', '를', '는', '으로']

# # 형태소 분석 및 불용어 제거하여 데이터 전처리하기
# tokenized_words = []
# sentence_nouns = []
# okt = Okt()

# # 각 문서에 대해 처리
# for document in korean_sample_data:
#     # 형태소 분석 및 불용어 제거
#     words = [word for word in okt.morphs(document, stem=True) if word not in stopwords]
    
#     # 형태소 추출 결과를 tokenized_words 리스트에 추가
#     tokenized_words.append(words)
    
#     # 명사 추출
#     nouns = [word for word, pos in okt.pos(document) if pos == 'Noun']
    
#     # 명사 리스트를 sentence_nouns 리스트에 추가
#     sentence_nouns.append(nouns)

# # 결과 출력
# print("tokenized_words:", tokenized_words)
# print("sentence_nouns:", sentence_nouns)


In [42]:
def compare_nouns(sample_data2):

    print(f'hananum: {hananum.nouns(sample_data2)}')
    print(f'kkma: {kkma.nouns(sample_data2)}')
    print(f'komoran: {komoran.nouns(sample_data2)}')
    print(f'okt: {okt.nouns(sample_data2)}')

### 기초 자연어 처리

1. 텍스트 데이터의 수를 기준(통계적 수치 확인)  
    1. 텍스트 길이의 최대, 최소, 중앙값, 평균값을 확인  
        - 최대값 : 임베딩을 할 때, max_len 파라미터에 대한 값을 설정
    2. 각 품사별 데이터 수 확인
        - 대부분은 명사가 많이 사용됨
        - 만약 하드웨어등의 사양이 낮아 문제가 되는 경우, 명사만 분석을 하기 위해 데이터 수 확인  
  
2. 용어(텍스트) 기준  
    1. 불용어 사용할 단어
    2. 유의어들 확인하기 위한 원문 데이터 확인

In [43]:
# 리뷰의 최대 길이
word_len = [len(l) for l in tokenized_words]
nouns_len = [len(l) for l in sentence_nouns]

print(f"가장 긴 단어 길이: {max(word_len)}, 가장 짧은 단어 길이: {min(word_len)}")
print(f"가장 긴 명사 길이: {max(nouns_len)}, 가장 짧은 명사 길이: {min(nouns_len)}")


'''
결과
가장 긴 단어 길이 : 95, 가장 짧은 단어 길이 : 1
가장 긴 명사 길이 : 66, 가장 짧은 명사 길이 : 0
'''

가장 긴 단어 길이: 60, 가장 짧은 단어 길이: 1
가장 긴 명사 길이: 29, 가장 짧은 명사 길이: 1


'\n결과\n가장 긴 단어 길이 : 95, 가장 짧은 단어 길이 : 1\n가장 긴 명사 길이 : 66, 가장 짧은 명사 길이 : 0\n'

## Word2Vec 훈련

- vector_size = 워드 벡터의 특징 값, 즉 임베딩 된 벡터의 차원  (128,512)
- window = 컨텐스트 윈도우 크기 (학습할 때 주변 단위를 얼마큼 볼 것이냐)
- min_count = 단어 최소 빈도 수 제한(빈도가 적은 단어들을 학습하지 않도록 하는 기준)  
- workers = 학습을 위한 프로세스 수 (workers로 설정을 하거나 혹은 multiprocessing으로 병렬처리)  
- sg = 0 -(CBOW) , 1 -(Skip-gram)  
- (Skip-gram)  이 성능이 높음 (중심단어를 통해 주변 단어를 예측하는 게 성능이 좋음)

In [44]:
# word2vec 모델 훈련하기
model_cbow = Word2Vec(tokenized_words, vector_size=100, window=5, min_count=5, workers=4, sg=0)
model_skipgram = Word2Vec(tokenized_words, vector_size=100, window=5, min_count=5, workers=4, sg=1)


In [45]:
# 완성된 임베딩 매트릭스의 크기 확인(단어수, 차원수)
print(model_cbow.wv.vectors.shape)
print(model_skipgram.wv.vectors.shape)
'''
결과
(15204, 100) 15204개의 단어를 100 차원으로 만듦
(15204, 100)
'''

(4, 100)
(4, 100)


'\n결과\n(15204, 100) 15204개의 단어를 100 차원으로 만듦\n(15204, 100)\n'

모델 저장 방법

1. 재훈련 없이 모델만 사용
2. 재훈련을 할 수 있도록 임베딩에 대한 기본 정보를 함께 저장

In [46]:
# 모델 저장 방법 1 : 재훈련을 하지 않는 경우
## Ram에 로드하지 않고도 디스크나 네트워크에서 데이터를 즉시 읽어 반복할 수 있음
## 모델 inference 에서 활용하면 좋음

model_cbow.wv.save_word2vec_format('./data/kor_w2v_cbow')
model_skipgram.wv.save_word2vec_format('./data/kor_w2v_skipgram')

In [54]:
# 모델 저장 방법1의 결과를 다시 로드해서 사용

from gensim.models import KeyedVectors

# 모델 불러오기
model_cbow_kv= KeyedVectors.load_word2vec_format('./data/kor_w2v_cbow')
model_skipgram_kv= KeyedVectors.load_word2vec_format('./data/kor_w2v_skipgram')

In [55]:
# 특정 단어를 중심으로 유사한 단어 확인하기
print(model_cbow_kv.most_similar("송강호", topn=5))
print(model_skipgram_kv.most_similar("송강호", topn=5))

[('차승원', 0.8818742632865906), ('김명민', 0.8706954121589661), ('김혜수', 0.8630266189575195), ('최민수', 0.8564978241920471), ('설경구', 0.8538017272949219)]
[('안성기', 0.8590131998062134), ('한석규', 0.8429045081138611), ('정재영', 0.838813841342926), ('황정민', 0.8353441953659058), ('류덕환', 0.8253893852233887)]


In [ ]:
# 모델 저장 방법2 : 재훈련 가능한 모델
model_cbow.save('./data/kor_w2v_cbow.model')
model_skipgram_kv.save('./data/kor_w2v_skipgram.model')


In [56]:
# 모델 불러오기
## 모델에 vocab 정보 등이 함께 저장된 결과를 로드함
model_cbow_model = Word2Vec.load('./data/kor_w2v_cbow.model')
model_skipgram_model = Word2Vec.load('./data/kor_w2v_skipgram.model')

In [58]:
# 특정 단어를 중심으로 유사한 단어 확인하기
print(model_cbow_model.wv.most_similar("송강호", topn=5))
print(model_skipgram_model.wv.most_similar("송강호", topn=5))

[('차승원', 0.8818742632865906), ('김명민', 0.8706954121589661), ('김혜수', 0.8630266189575195), ('최민수', 0.8564978241920471), ('설경구', 0.8538017272949219)]
[('안성기', 0.8590131998062134), ('한석규', 0.8429045081138611), ('정재영', 0.838813841342926), ('황정민', 0.8353441953659058), ('류덕환', 0.8253893852233887)]


In [63]:
# 두 단어 간의 유사도 파악하기 - 코사인 유사도
model_skipgram_model.wv.similarity("추천", "강력")

model_skipgram_model.wv.similarity("망작", "쓰레기")
model_skipgram_model.wv.similarity("노잼", "쓰레기")

0.47882542

In [68]:
# 사전에 없는 경우 확인
model_skipgram_model.wv.most_similar("최고")

[('최고다', 0.772616446018219),
 ('쵝오', 0.7664356827735901),
 ('단연', 0.7442367076873779),
 ('손꼽다', 0.718795657157898),
 ('꼽는', 0.7099205255508423),
 ('최강', 0.709646999835968),
 ('으뜸', 0.6837571859359741),
 ('최고봉', 0.6809198260307312),
 ('쵝', 0.6775249242782593),
 ('연애시대', 0.6722466349601746)]

In [71]:
# 여러 단어와 유사한 embedding 가져오기
model_skipgram_model.wv.most_similar(positive=["여자", "감독"]) # 여자, 감독과 유사한 임베딩
model_skipgram_model.wv.most_similar(positive=["여자", "감독"] , negative=["남자"])



model_skipgram_model.wv.most_similar(positive=["노잼", "쓰레기"] )


[('노답', 0.7824556827545166),
 ('렉', 0.7778365612030029),
 ('좆', 0.777439296245575),
 ('개핵', 0.7581625580787659),
 ('핵', 0.7469645142555237),
 ('짱깨', 0.7467575669288635),
 ('극혐', 0.7430484294891357),
 ('씨발', 0.736915647983551),
 ('날림', 0.7270280718803406),
 ('절때', 0.7269129753112793)]

In [75]:
# cosmul 사용해 보기
model_skipgram_model.wv.most_similar_cosmul(positive=['남자', '감독'], negative=['여자'])
model_skipgram_model.wv.most_similar_cosmul(positive=['눈물', '슬픔'])

[('울음', 0.7214616537094116),
 ('주룩주룩', 0.7146351933479309),
 ('코끝', 0.7118856906890869),
 ('폭풍눈물', 0.7094402313232422),
 ('주르륵', 0.7090370059013367),
 ('멎다', 0.7052245736122131),
 ('회한', 0.6988445520401001),
 ('펑펑', 0.69825679063797),
 ('방울', 0.6977316737174988),
 ('하염없이', 0.6957030892372131)]

In [84]:
# 그룹간의 유사도 측정
print(model_skipgram_model.wv.n_similarity(['남자', '배우'], ['여자', '감독']))
print(model_skipgram_model.wv.n_similarity(['콧물', '눈물'], ['펑펑', '울음']))
print(model_skipgram_model.wv.n_similarity(['노잼', '쓰레기'], ['존잼', '꿀잼']))




0.6814057
0.870179
0.5506598


In [83]:
# 가장 유사하지 않은 단어를 추출
model_skipgram_model.wv.doesnt_match(['영화', '밥', '국밥', '소머리국밥'])

'밥'

기존에 학습된 모델 확인 및 사용

In [86]:
import gensim.downloader as api
from pprint import pprint as pp


pp(list(gensim.downloader.info()['models'].keys()))


['fasttext-wiki-news-subwords-300',
 'conceptnet-numberbatch-17-06-300',
 'word2vec-ruscorpora-300',
 'word2vec-google-news-300',
 'glove-wiki-gigaword-50',
 'glove-wiki-gigaword-100',
 'glove-wiki-gigaword-200',
 'glove-wiki-gigaword-300',
 'glove-twitter-25',
 'glove-twitter-50',
 'glove-twitter-100',
 'glove-twitter-200',
 '__testing_word2vec-matrix-synopsis']


In [87]:
# vector load
glove_vector_25= api.load("glove-twitter-25")

[==================================================] 100.0% 104.8/104.8MB downloaded


In [93]:
# 유사도 확인하기
glove_vector_25.most_similar("instagram")

[('tumblr', 0.9367011189460754),
 ('facebook', 0.9184155464172363),
 ('insta', 0.9175950884819031),
 ('twitter', 0.9104822278022766),
 ('youtube', 0.9031587839126587),
 ('skype', 0.8910087943077087),
 ('hashtag', 0.8861388564109802),
 ('spam', 0.8861327171325684),
 ('link', 0.8775855302810669),
 ('chat', 0.8751716017723083)]

In [94]:
glove_vector_100= api.load("glove-twitter-100")

[==================================================] 100.0% 387.1/387.1MB downloaded


In [98]:
glove_vector_100.most_similar("happy")

[('birthday', 0.9259342551231384),
 ('day', 0.8549740314483643),
 ('bday', 0.8145427107810974),
 ('merry', 0.7886534333229065),
 ('love', 0.7855904698371887),
 ('wish', 0.7771798372268677),
 ('hope', 0.7660956978797913),
 ('thank', 0.7639737129211426),
 ('year', 0.7505833506584167),
 ('thanks', 0.7475083470344543)]